## Building A Chatbot

<div dir='rtl'>

في هذا الـ Notebook، سنتعلم كيفية تصميم وتنفيذ **شات بوت مدعوم بنموذج لغة كبير (LLM)**.  
هذا الشات بوت سيكون قادرًا على إجراء محادثة وتذكر التفاعلات السابقة.

---

### ملاحظات مهمة:

الشات بوت الذي سنبنيه هنا سيستخدم فقط نموذج اللغة لإجراء المحادثة.  
ولكن هناك عدة مفاهيم متعلقة قد تود الاطلاع عليها أيضًا:

- **Conversational RAG**: تمكين تجربة شات بوت تعتمد على مصدر خارجي للبيانات.
- **Agents**: بناء شات بوت يمكنه اتخاذ إجراءات بناءً على المحادثة.

---

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [ ]:
groq_api_key = os.getenv('GROQ_API_KEY')
from langchain_groq import ChatGroq
model = ChatGroq(model = 'Gemma2-9b-It', groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002CC280DE660>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002CC280DF7A0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = 'Hi, My Name Is Ahmed Sharaf and I am a ai engineer')])

AIMessage(content="Hello Ahmed Sharaf, it's nice to meet you! \n\nThat's fantastic that you're an AI engineer. It's a really exciting field right now.  \n\nIs there anything I can help you with today? Perhaps you'd like to discuss some AI concepts, explore a specific project, or just chat about the latest developments in the field?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 23, 'total_tokens': 102, 'completion_time': 0.143636364, 'prompt_time': 0.002135306, 'queue_time': 0.157774412, 'total_time': 0.14577167}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d2030072-089f-40b9-bc77-be34f345a767-0', usage_metadata={'input_tokens': 23, 'output_tokens': 79, 'total_tokens': 102})

In [ ]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content='Hi, My Name Is Ahmed Sharaf and I am a ai engineer'),
        AIMessage(content = "Hello Ahmed Sharaf, it's nice to meet you! \n\nThat's fantastic that you're an AI engineer. It's a really exciting field right now"), # هنا بقوله ان دا كان رد ال Ai 
        HumanMessage(content = "Hey What's my name and what do i do ?")
    ]
)

# هنا قدر انه يفتكر المعلومات القديمه بس احنا ادينهاله يدويا بنفسنا
# ازاى نعمل الكلام دا تلقائي

AIMessage(content="You told me your name is Ahmed Sharaf and that you are an AI engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 81, 'total_tokens': 130, 'completion_time': 0.089090909, 'prompt_time': 0.004072432, 'queue_time': 0.16206112900000003, 'total_time': 0.093163341}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--f150d84a-d967-485c-9dbe-b683ba61ee0a-0', usage_metadata={'input_tokens': 81, 'output_tokens': 49, 'total_tokens': 130})

### Message History
<div dir='rtl'>
نستطيع استخدام **كلاس Message History** لتغليف النموذج وجعله **يحفظ الحالة (stateful)**.  
هذا يعني أنه سيقوم بتتبع المدخلات والمخرجات الخاصة بالنموذج، ويخزنها في قاعدة بيانات أو مكان تخزين معين.
datastore
وبالتالي، عند التفاعل في المستقبل، يتم تحميل هذه الرسائل السابقة وتمريرها إلى الـ chain كجزء من الإدخال، مما يسمح للنموذج بفهم سياق المحادثة السابقة.

---

دعونا نرى كيفية استخدام هذا المفهوم عمليًا!


In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# اول حاجه ازاى نفرق بين السيشنز المختلفه يعني بين المحادثات المختلفه
# دا عن طريق داله بتوشف ال session id
# ChatMessageHistory => دا فيه المحادثه كامله بالتاريخ

# session_id type => string
# -> return type
store = dict()
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# علشان اخليه يشغ الداله دي ف كل مره بنكلم الموديل ونخزن المحتوى هنا
# بنستخدم RunnableWithMessageHistory
# دا اللى بيكون ال pipline

work_message_history = RunnableWithMessageHistory(model, get_session_history)


In [13]:
config = {'configurable': {'session_id':'chat1'}}
response = work_message_history.invoke(
    [HumanMessage(content='Hi, My Name Is Ahmed Sharaf and I am a ai engineer')],
    config = config,
)
response.content

"Hi Ahmed! It's great to meet you. \n\nBeing an AI engineer is such a cutting-edge field. What areas of AI are you most interested in or working on? I'm always curious to hear about the exciting projects people are involved in. 😊  \n"

In [14]:
work_message_history.invoke(
    [HumanMessage(content='What is my name?')],
    config = config,
)

AIMessage(content='Your name is Ahmed Sharaf.  😊 \n\nI remembered it from our first introduction!  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 174, 'total_tokens': 198, 'completion_time': 0.043636364, 'prompt_time': 0.007277485, 'queue_time': 0.1595678, 'total_time': 0.050913849}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cd205b2b-42f2-4d94-bb2e-90e77aa37a7b-0', usage_metadata={'input_tokens': 174, 'output_tokens': 24, 'total_tokens': 198})

In [16]:
# if we change the config (session id) the model can't remeber me
config1 = {'configurable': {'session_id': 'chat2'}}
work_message_history.invoke(
    [HumanMessage(content='What is my name?')],
    config = config1,
)

AIMessage(content="As a large language model, I have no memory of past conversations or access to personal information. Therefore, I don't know your name.\n\nIf you'd like to tell me your name, I'd be happy to know!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 14, 'total_tokens': 66, 'completion_time': 0.094545455, 'prompt_time': 0.001914866, 'queue_time': 0.16107137500000002, 'total_time': 0.096460321}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--74d83f09-0b61-47ad-a72e-b16d05590bda-0', usage_metadata={'input_tokens': 14, 'output_tokens': 52, 'total_tokens': 66})

In [17]:
work_message_history.invoke(
    [HumanMessage(content='My Name Ali')],
    config = config1,
)

AIMessage(content="Hi Ali, it's nice to meet you! 😊\n\nIs there anything I can help you with today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_time': 0.047272727, 'prompt_time': 0.003847522, 'queue_time': 0.159495923, 'total_time': 0.051120249}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4c7165a5-62d9-4350-954a-ed857571f832-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103})

In [18]:
work_message_history.invoke(
    [HumanMessage(content='whats my name?')],
    config = config1,
)

AIMessage(content='Your name is Ali.  \n\nI remember that you told me earlier!  😄  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 115, 'total_tokens': 143, 'completion_time': 0.050909091, 'prompt_time': 0.005832969, 'queue_time': 0.160230962, 'total_time': 0.05674206}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--7d8aea32-c238-4658-8edf-442bdc6437c2-0', usage_metadata={'input_tokens': 115, 'output_tokens': 28, 'total_tokens': 143})

<div dir="rtl">

# ✅ ملخص وظائف:
| الكلاس | وظيفته | علاقته بالذاكرة (Memory) |
|---------|---------|-------------------------|
| **ChatMessageHistory** | كلاس مسؤول عن تخزين واسترجاع تاريخ الرسائل (User + AI) داخل المحادثة | بيحتفظ بالرسائل في الذاكرة (in-memory) أو ممكن يتخزن في قواعد بيانات زي Redis |
| **BaseChatMessageHistory** | كلاس مجرد (Abstract Base Class) بيحدد الواجهة الأساسية لأي Message History | مش بيخزن حاجة بنفسه، لكن بيحدد القواعد (interface) اللي ChatMessageHistory وغيره لازم يلتزموا بيها |
| **RunnableWithMessageHistory** | Wrapper بيربط الـ Chain أو Runnable بتاريخ المحادثة تلقائي | بياخد الرسائل من الـ history، يمررها للموديل، وبعد الرد بيحدث التاريخ بدون ما انت تتدخل |

---

## ✅ باختصار:
- **ChatMessageHistory** = التخزين الفعلي للرسائل في الذاكرة.
- **BaseChatMessageHistory** = القالب اللي كل أنواع الـ history لازم تمشي عليه.
- **RunnableWithMessageHistory** = الكائن اللي بيربط الـ Chain مع التاريخ تلقائي ويخليك تنسى التفاصيل.

</div>


### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

<div dir="rtl">

# ✅ ما هو MessagesPlaceholder في LangChain؟

### 🟡 التعريف:
- `MessagesPlaceholder` هو **مكون (Component)** بيمثل **مكان مخصص** داخل الـ Prompt Template.
- وظيفته إنه يكون مكان (Placeholder) نقدر نحط فيه ** الرسائل (Message)** بعدين.

---

## ✅ ليه بنستخدمه؟
| السبب | الشرح |
|--------|-------|
| **عشان ندخل التاريخ في البرومبت بسهولة** | بدل ما نكتب الرسائل يدوي جوا البرومبت، بنحط MessagesPlaceholder وهو بيستقبلها وقت التنفيذ. |
| **يحافظ على ترتيب الرسائل (User + AI)** | بيضيف الرسائل بالترتيب الصح في الـ input بتاع الموديل. |
| **بيشتغل مع RunnableWithMessageHistory** | لما تستخدمهم مع بعض، الـ Placeholder ده بيتملّى تلقائي بالتاريخ اللي محفوظ. |
---



In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You Are a helpful assistant. Answer all the question to the nest of your ability'),
        MessagesPlaceholder(variable_name = 'messages')
    ]
)
chain = prompt|model


In [23]:
chain.invoke({'messages':[HumanMessage(content = 'Hi My Name Is Ahmed Sharaf')]})

AIMessage(content="Hello Ahmed Sharaf, it's nice to meet you!\n\nI'm ready to answer your questions to the best of my ability.  What can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 32, 'total_tokens': 76, 'completion_time': 0.08, 'prompt_time': 0.002365776, 'queue_time': 0.16124811700000002, 'total_time': 0.082365776}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cca7a0b7-aaeb-40da-9f90-c99cf10af8f7-0', usage_metadata={'input_tokens': 32, 'output_tokens': 44, 'total_tokens': 76})

In [24]:
# How Invoke with messages history
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [37]:
config3 = {'configurable': {'session_id': "chat4"}}
respone = with_message_history.invoke(
    [HumanMessage(content = 'Hi My Name Is Ahmed Sharaf')],
    config=config3
)

In [38]:
respone

AIMessage(content="Hello Ahmed Sharaf, it's nice to meet you! \n\nI'm happy to help with any questions you have.  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 32, 'total_tokens': 75, 'completion_time': 0.078181818, 'prompt_time': 0.002337925, 'queue_time': 0.161324813, 'total_time': 0.080519743}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ff84c9a8-ca5e-4aae-812c-d7408f174e19-0', usage_metadata={'input_tokens': 32, 'output_tokens': 43, 'total_tokens': 75})

In [39]:
respone = with_message_history.invoke(
    [HumanMessage(content = "what's my name?")],
    config=config3
)
response.content

"Hi Ahmed! It's great to meet you. \n\nBeing an AI engineer is such a cutting-edge field. What areas of AI are you most interested in or working on? I'm always curious to hear about the exciting projects people are involved in. 😊  \n"

In [44]:
# Lets add multiple variable
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful assistant. Answer all Question to the best of your ability in {language}'),
        MessagesPlaceholder(variable_name='messages')
    ]
)

chain = prompt|model

In [45]:
respone = chain.invoke({'messages': [HumanMessage(content="Hi my name is ahmed")], 'language': 'arabic'})

In [46]:
respone

AIMessage(content='مرحباً أحمد، يسعدني التعرّف عليك! \n\nكيف يمكنني مساعدتك اليوم؟  😊 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 32, 'total_tokens': 62, 'completion_time': 0.054545455, 'prompt_time': 0.002398085, 'queue_time': 0.161231276, 'total_time': 0.05694354}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--aadfd6c3-6f9b-4fa6-abce-5df44468f1c4-0', usage_metadata={'input_tokens': 32, 'output_tokens': 30, 'total_tokens': 62})

In [ ]:
with_message_history = RunnableWithMessageHistory(chain, 
                                                  get_session_history,
                                                  input_messages_key='messages')

# input_messages_key => هنا بتعرفه على ال messages اللى هتدخله 
# لاحظ اى متعير ف ال prompt بندخل قيمته هنا ب dictionary 
# key => اسم المتغير ف ال prompt
# value => القيمه اللى هيعوض بيها مكانه ف ال prompt

In [ ]:
config_5 = {'configurable': {'session_id': 'chat10'}}
response = with_message_history.invoke(
    {'messages': [HumanMessage(content='Hi, I am ahmed')],
     'language': 'Arabic'},
    config=config_5
)
response.content

"مرحبا أحمد، يسعدني مساعدتك!  \n\n(Hello Ahmed, I'm happy to help you! )\n\nما الذي يمكنني أن أفعل لك؟ \n\n(What can I do for you?) \n\n"

In [49]:
response = with_message_history.invoke(
    {'messages': [HumanMessage(content='what is my name?')],
     'language': 'Arabic'},
    config=config_5
)
response.content

'اسمك أحمد. 😊 \n\n(Your name is Ahmed. 😊) \n\n\n\n\n'

<div dir="rtl">

# ✅ إدارة تاريخ المحادثة (Managing Conversation History)

### 🟡 المشكلة:
- كل مرة بنتكلم مع الشات بوت، الرسائل (user + AI) بتضاف لتاريخ المحادثة.
- مع الوقت، التاريخ بيكبر جدًا وممكن:
  - **يتجاوز الـ Context Window** للنموذج (يعني الموديل مش هيقدر يشوف كل الرسائل).
  - يبطّأ الأداء ويستهلك Tokens زيادة.

---

## ✅ الحل: استخدام **Trimmer**
- هو أداة (Helper Function) بتساعد في **تقليل حجم الرسائل** اللي بنبعتها للموديل.
- الهدف إننا نفضل محافظين على **أهم الرسائل** بس، والباقي بنرميه أو بنقصه.

---

## ✅ وظيفة `trim_messages`:
| الخاصية | شرحها |
|----------|-------|
| **تحديد عدد الـ Tokens اللي عايزين نحافظ عليهم** | بنقول له مثلًا: "ما تتعداش 2000 token" |
| **إبقاء رسالة الـ System دايمًا** | عشان توجه النموذج دايمًا مهما قصينا من التاريخ. |
| **السماح بالـ Partial Messages** | يعني ممكن نقص جزء من رسالة بدل ما نمسحها كلها. |

---

## ✅ ليه مهم؟
- بيحميك من **Overflow** في الـ Context.
- بيحافظ على **السياق الأهم** (recent messages + system prompt).
- بيخلي الشات بوت **شغال بكفاءة** حتى مع جلسات طويلة.

---

In [52]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 45,
    strategy = 'last', # الطريقه اللى هيعرف بيها ان دي اهم جزء من الرسائل فيحافظ عليه ويرمي الباقي
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = 'human'
)

# بنينا محادثه بين انسان وذكاء اصطناعى
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

## ✅ Trimmer Parameters

<div dir="rtl">

| البراميتر               | الوظيفة                                    | التفسير                                                   |
|------------------------|--------------------------------------------|-----------------------------------------------------------|
| `max_tokens = 45`       | أقصى عدد من الـ Tokens بعد التريمنج        | الموديل هيشوف رسائل مجموعهم لا يتجاوز 45 Token فقط         |
| `strategy = 'last'`     | استراتيجية اختيار الرسائل                 | هيحتفظ بآخر الرسائل (الأحدث) ويقص القديمة                  |
| `token_counter = model` | أداة لحساب عدد الـ Tokens في الرسائل       | بيستخدم الموديل نفسه لحساب حجم الرسائل بالتوكنز           |
| `include_system = True` | هل نحافظ على system message مهما حصل؟     | نعم، هتفضل رسالة الـ system موجودة حتى بعد القص           |
| `allow_partial = False` | السماح بقص جزء من رسالة؟                  | لأ، إما الرسالة تفضل كاملة أو تتشال خالص                   |
| `start_on = 'human'`    | منين نبدأ التريمنج؟                       | هيبدأ من أول رسالة بشرية (Human Message)                   |

</div>


In [54]:
# how do pass trimmer in chain ?
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    | prompt
    | model
)
response = chain.invoke(
    {
    'messages': messages + [HumanMessage(content = 'What ice cream do i like')],
    'language': 'Arabic'
    }
)
response.content

'أنا مساعد افتراضي ولا أعرف تفضيلاتك الشخصية، بما في ذلك نوع الآيس كريم الذي تحبه.  \n\nهل يمكنك مشاركة نوع الآيس كريم الذي تفضله؟ 😊 \n\n'

-  هنا مفتكرش علشان ال trimmer مخدش الجزء دا ف المهم اللى بعته له 

In [ ]:
response = chain.invoke(
    {
    'messages': messages + [HumanMessage(content = 'What math problem did i ask')],
    'language': 'Arabic'
    }
)
response.content

'سألتني 2 + 2  \n'

In [63]:
# Lets Wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)
config = {'configurable': {'session_id': 'chat100'}}
response = with_message_history.invoke(
    {
        'messages': messages + [HumanMessage(content = "what's my name")],
        'language': 'English'
    },
    config=config
)

response.content

"As an AI, I don't have memory of past conversations or personal information about you.  So I don't know your name. Would you like to tell me? 😊  \n\n\n\n\n"